In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# load data
merged_df = pd.read_csv("Data/final_modeling_dataset.csv")
# Check columns.
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   periodname              349 non-null    object 
 1   week_start              349 non-null    object 
 2   week_end                349 non-null    object 
 3   Combined_positive       349 non-null    float64
 4   temp_c                  349 non-null    float64
 5   rh_pct                  349 non-null    float64
 6   rain_mm                 349 non-null    float64
 7   wind10_kmh              349 non-null    float64
 8   soil_moisture_top_m3m3  349 non-null    float64
dtypes: float64(6), object(3)
memory usage: 24.7+ KB


### 📌 Step 1: Data Preparation  
#### 1.1 Convert Dates and Set Index

In [3]:
merged_df['week_start'] = pd.to_datetime(merged_df['week_start'])
merged_df.set_index('week_start', inplace=True)
merged_df.sort_index(inplace=True)

#### 1.2 Check Stationarity (ADF Test)

In [4]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(merged_df['Combined_positive'])
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')

ADF Statistic: -5.497344209946364
p-value: 2.108878552986943e-06


If p-value > 0.05, the series is non-stationary. You’ll likely need d=1.

**Code output and interpretation**  
ADF Statistic: -5.50
p-value: 2.11e-06  
Since p-value < 0.05, the null hypothesis of non-stationarity is rejected.  
✅ This means the Combined_positive series is already stationary, and we don’t need differencing (d=0) for SARIMAX.

### 📌 Step 2: Build a Baseline SARIMAX Model

In [5]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

exog_vars = ['temp_c', 'rh_pct', 'rain_mm', 'wind10_kmh', 'soil_moisture_top_m3m3']
exog = merged_df[exog_vars]

model = SARIMAX(merged_df['Combined_positive'], 
                exog=exog,
                order=(1, 0, 1), 
                seasonal_order=(1, 0, 1, 52),  # 52 weeks for annual seasonality
                enforce_stationarity=False,
                enforce_invertibility=False)

results = model.fit(disp=False)
print(results.summary())

C:\Users\HRumenya\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
C:\Users\HRumenya\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
C:\Users\HRumenya\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                     SARIMAX Results                                      
Dep. Variable:                  Combined_positive   No. Observations:                  349
Model:             SARIMAX(1, 0, 1)x(1, 0, 1, 52)   Log Likelihood               -2443.876
Date:                            Sun, 21 Sep 2025   AIC                           4907.753
Time:                                    17:37:10   BIC                           4944.622
Sample:                                12-31-2018   HQIC                          4922.516
                                     - 09-01-2025                                         
Covariance Type:                              opg                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
temp_c                   146.3711     25.134      5.824      0.000      97.109     195.633

### 📌 Step 3: Tune SARIMAX Hyperparameters
Use Grid Search to Improve Performance

In [6]:
import itertools
import warnings
warnings.filterwarnings("ignore")

p = q = range(0, 2)
d = [0]  # Keep d=0 based on ADF test
seasonal_d = [0]  # Keep D=0 based on ADF test

# Generate all combinations for trend and seasonal parameters
pdq = list(itertools.product(p, d, q))
seasonal_pdq = list(itertools.product(p, seasonal_d, q))

best_aic = float("inf")
best_order = None
best_seasonal_order = None

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = SARIMAX(merged_df['Combined_positive'],
                          exog=exog,
                          order=param,
                          seasonal_order=param_seasonal + (52,),
                          enforce_stationarity=False,
                          enforce_invertibility=False)

            results = mod.fit(disp=False)

            if results.aic < best_aic:
                best_aic = results.aic
                best_order = param
                best_seasonal_order = param_seasonal

        except:
            continue

print(f'Best SARIMA order: {best_order}')
print(f'Best seasonal order: {best_seasonal_order} (52)')
print(f'Best AIC: {best_aic}')

Best SARIMA order: (1, 0, 1)
Best seasonal order: (0, 0, 1) (52)
Best AIC: 4888.447874768525


### 📌 Step 4: Train Final Tuned Model

In [7]:
final_model = SARIMAX(merged_df['Combined_positive'],
                      exog=exog,
                      order=best_order,
                      seasonal_order=best_seasonal_order + (52,),
                      enforce_stationarity=False,
                      enforce_invertibility=False)

final_results = final_model.fit(disp=False)
print(final_results.summary())

                                     SARIMAX Results                                      
Dep. Variable:                  Combined_positive   No. Observations:                  349
Model:             SARIMAX(1, 0, 1)x(0, 0, 1, 52)   Log Likelihood               -2435.224
Date:                            Sun, 21 Sep 2025   AIC                           4888.448
Time:                                    17:41:55   BIC                           4921.631
Sample:                                12-31-2018   HQIC                          4901.735
                                     - 09-01-2025                                         
Covariance Type:                              opg                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
temp_c                   169.2506     36.830      4.595      0.000      97.065     241.436

### 📌 Step 5: Evaluate the Model 
**Train-Test Split & Forecast Accuracy**

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Split data (last 20 weeks as test)
train = merged_df.iloc[:-20]
test = merged_df.iloc[-20:]

train_exog = train[exog_vars]
test_exog = test[exog_vars]

model = SARIMAX(train['Combined_positive'],
                exog=train_exog,
                order=best_order,
                seasonal_order=best_seasonal_order + (52,),
                enforce_stationarity=False,
                enforce_invertibility=False)

results = model.fit(disp=False)

# Forecast
forecast = results.predict(start=test.index[0], end=test.index[-1], exog=test_exog)

# Evaluation
mae = mean_absolute_error(test['Combined_positive'], forecast)
rmse = np.sqrt(mean_squared_error(test['Combined_positive'], forecast))
r2 = r2_score(test['Combined_positive'], forecast)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2 Score: {r2:.2f}")

MAE: 1269.78
RMSE: 1789.21
R2 Score: -0.07


### 📌 Step 6: Forecast 5 Weeks Ahead  
#### 6.1 Prepare Future Exog (you must update with actual values or use last known)

In [9]:
# Assuming last 5 weeks of known exog values are representative
future_exog = merged_df[exog_vars].iloc[-5:].copy()
future_exog.index = pd.date_range(start=merged_df.index[-1] + pd.Timedelta(weeks=1), periods=5, freq='W-MON')

forecast_5w = final_results.predict(start=future_exog.index[0], 
                                    end=future_exog.index[-1],
                                    exog=future_exog)

print(forecast_5w)

2025-09-08    3039.547826
2025-09-15    2841.421373
2025-09-22    2665.092456
2025-09-29    2847.600838
2025-10-06    2735.464744
Freq: W-MON, Name: predicted_mean, dtype: float64


### 📌 Step 7: Detect Outbreaks Based on Rule

In [10]:
# Detect outbreaks: pred(t+1)/pred(t) > 1.5
def detect_outbreaks(pred_series):
    outbreaks = []
    for i in range(1, len(pred_series)):
        ratio = pred_series.iloc[i] / pred_series.iloc[i - 1]
        is_outbreak = ratio > 1.5
        outbreaks.append((pred_series.index[i], pred_series.iloc[i], ratio, is_outbreak))
    return outbreaks

outbreaks = detect_outbreaks(forecast_5w)

for date, value, ratio, is_outbreak in outbreaks:
    print(f"Week of {date.date()} | Cases: {value:.2f} | Ratio: {ratio:.2f} | Outbreak: {'Yes' if is_outbreak else 'No'}")

Week of 2025-09-15 | Cases: 2841.42 | Ratio: 0.93 | Outbreak: No
Week of 2025-09-22 | Cases: 2665.09 | Ratio: 0.94 | Outbreak: No
Week of 2025-09-29 | Cases: 2847.60 | Ratio: 1.07 | Outbreak: No
Week of 2025-10-06 | Cases: 2735.46 | Ratio: 0.96 | Outbreak: No
